# ベイジアン合成コントロール法

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pymc as pm
import arviz as az
az.style.use('arviz-darkgrid')

In [2]:
path = '../../../../data/processed/df_exclude_already_treated.xlsx'
df = pd.read_excel(path)
df['log_pop'] = np.log(df['population'])

## モデル

$$
y_{it} = \mathbf{X}^T \mathbf{\beta} + \epsilon_{it}
$$

In [6]:
with pm.Model() as model:
    # prior
    alpha = pm.Laplace('alpha', mu=0, b=1)
    # beta = pm.Laplace('beta', mu=0, b=1)
    sigma = pm.HalfCauchy('sigma', beta=1)
    island_effect = pm.Laplace('island_effect', mu=0, b=1, shape=len(df['island_ids'].unique()))
    year_effect = pm.Laplace('year_effect', mu=0, b=1, shape=len(df['year'].unique()))

    # likelihood
    mu = alpha + island_effect[df['island_ids']] + year_effect * df['year']

    y = pm.Normal('y', mu=mu, sigma=sigma, observed=df['log_pop'])

pm.model_to_graphviz(model)

ValueError: Incompatible Elemwise input shapes [(29,), (1707,)]